This is series of image anlysis functions that I have been pulling together over the last little while in no particular order

# Handling data

## Read in multidimensional .nd2 file

In [6]:
'''read_multidim_nd2 : last updated 06-15-20
This is function to read in a .nd2 file that is multideminsional containing multiple fields of view. The images can be either zstacks
or single slices. Right now this assumes that the .nd2 contains multiple fieds of view and multiple channels. There is the beginnings of 
a more robust version of this that can handle .nd2 files with different data structures. Heads up this takes a little bit of time to 
run as it reads the entire file into RAM

parameters
-------------
filename : .nd2 file
    this can be single or multiple fields of view, single or multiple channel, single or multiple z-slice
final_dtype: string datatype
    the data type of the final numpy array. This defaults to 16-bi integer

Returns
------------------------
array_multidim: ndarray
    this is multidimensional array. The order of the dimensions will depend on structure of .nd2 image but if defaults are all left the 
    same the order will be fov, ch, z, y, x'''
    
from nd2reader import ND2Reader
import numpy as np
    
def read_multidim_nd2(filename, final_dtype = 'uint16'):
    nd2_multidim = ND2Reader(filename) # read in a vdr-gr-cebpb image that has no dapi
    if 'z' in nd2_multidim.sizes:
        nd2_multidim.bundle_axes = 'czyx' #bundle the channel and zyx coordinates
        nd2_multidim.iter_axes = 'v' # set the index to the field of view
    else:
        nd2_multidim.bundle_axes = 'cyx' #bundle the channel and zyx coordinates
        nd2_multidim.iter_axes = 'v' # set the index to the field of view
    array_multidim = np.array(nd2_multidim, dtype = final_dtype) #change the pims image into a numpy array containing float
    return(array_multidim)


In [ ]:
############################################################################################################################
#Work in progress function to convert .nd2 files into multideimsional nparray. The skeleton is here already but it is just going to 
#take either some chop wood carry water kind of scripting or some other clever way determine the structure of the input data
###############################################################################################################################33

'''This is function to read in a .nd2 file that is multideminsional containing multiple fields of view and/or multiple channels and/or
multiple z-slices. I haven't taken the time to make to amend for time series as it is complicated to deal with the hierarchy of dimensions
given many different data structurs but but with a little help it it should be largely convertable to make it handle this as well. It 
takes a second to work as it reads the file into ram. It reads in a .nd2 file and returns a multidemensional numpy array.

parameters
-------------
filename : .nd2 file
    this can be single or multiple fields of view, single or multiple channel, single or multiple z-slices
final_dtype: string datatype
    the data type of the final numpy array. This defaults to 16-bi integer

Returns
------------------------
array_multidim: ndarray
    this is multidimensional array. The order of the dimensions will depend on structure of .nd2 image but if defaults are all left the 
    same the order will be fov, ch, z, y, x
'''

from nd2reader import ND2Reader
import numpy as np



def read_in_nd2(filename, final_dtype = 'uint16'):
    nd2_multidim = ND2Reader(filename) # read in a vdr-gr-cebpb image that has no dapi
    if 'v' not in nd2_multidim.sizes: #this would be image of a single field of view       
        if 'z' not in nd2_multidim.sizes: # this would be a multichannel images of single focal plances
            nd2_multidim.bundle_axes = 'yx' # bundle yx coordinates
            nd2_multidim.iter_axes = 'c' # set the index to the channel
        if 'c' not in nd2_multidim.sizes: #this would be single channle z-stack
            nd2_multidim.bundle_axes = 'yx' # yx coordinates
            nd2_multidim.iter_axes = 'z' # set the index to the z
        else: # this would be single field of view, multichannel z-stack
            nd2_multidim.bundle_axes = 'zyx' # zyx coordinates
            nd2_multidim.iter_axes = 'c' # set the index to the channel
    if 'c' not in nd2_multidim.sizes: #right now this assumes that image single channel, multiple fovs and 
        nd2_multidim.bundle_axes = 'zyx' # channel and yx coordinates
        nd2_multidim.iter_axes = 'v' # set the index to the channel
    if 'z' not in nd2_multidim.sizes:
        nd2_multidim.bundle_axes = 'cyx' # channel and yx coordinates
        nd2_multidim.iter_axes = 'v' # set the index to the channel
    else:
        nd2_multidim.bundle_axes = 'czyx' #bundle the channel and zyx coordinates
        nd2_multidim.iter_axes = 'v' # set the index to the field of view
    array_5d = np.array(nd2_multidim, dtype = final_dtype) #change the pims image into a numpy array containing float

## make z-stack from folder of .tif files

In [55]:
'''make_z_stack: updated :06-15-20

This is function for making a multihannel z-stack from from a folder containing sequential .tif files.  
This is for making z-stacks from a folder that contains individual tiffs for each channel and z-slice. It also assumes that the naming
conventions that are spit out of elements. i.e. z01c1, z01c2.... z##c#.  It will work on up 5 channels 

Parameters
--------------
tif_directory: string
    This is the directory that contains the .tif files. This should be absent of other files and in the proper order. The default
    value for this is the current working directory in order to accomodate some previous scripts in which I moved to the .tif 
    directory in the script itself
final_dtype: string
    this is the data type for the final output nparray

Returns
--------------
img_arr: nparray
    This is a numpy array that is the final z-stack all put into one array

'''
import os
import numpy as np
from skimage import io

def make_z_stack(tif_directory = os.getcwd(), final_dtype = 'uint16'):
    orig_dir = os.getcwd() #keep traick of working directoyr prior to moving to directory with .tif files
    os.chdir(tif_directory) #change to directory with .tif files 
    
    if 'Thumbs.db' in os.listdir(): #windows likes to throw these stupid files seemingly randomnly into folders during iconf view of the foler
        os.remve('Thumbs.db') #they are basically useless to have around so it is fine to just get rid of it
        
    os.listdir().sort() #this is probably over kill but this code is a little fragile WRT to the order of the files
    
    '''determine the number of channles that this zstack hase. '''
    channel_lst = [] # list to be populated with the channel numbers of the first six files in the folder
    for filename in os.listdir()[:6]: #iterate over the first 6 files. Shouldn't ever be a need for more than five channels
        channel_lst.append(int(filename[-5])) # append the list with the channel number derived from the filename
    num_channels = max(channel_lst)
    ###generate list of sorted images
    image_lst = np.sort(os.listdir())
    ### figure out shape of empty zeros array for image
    img_ = io.imread(image_lst[0])
    n_row = img_.shape[0]
    n_col = img_.shape[1]
    n_plane = int(len(image_lst) / num_channels)
    #shape_tupple = np.zeros((n_z, n_row, n_col))
    
    ### generate some empyt arrays to be filled up
    c1_img = np.zeros((n_plane, n_row, n_col))
    c2_img = np.zeros((n_plane, n_row, n_col))
    c3_img = np.zeros((n_plane, n_row, n_col))
    c4_img = np.zeros((n_plane, n_row, n_col))
    c5_img = np.zeros((n_plane, n_row, n_col))
    
    ###read in and generate z-stack for each channel
    z_slice = 0
    for i in range(0 , (len(image_lst))- num_channels, num_channels):
        if num_channels == 1:
            c1_img[z_slice, :, :] = io.imread(image_lst[i])
        if num_channels == 2:
            c1_img[z_slice, :, :] = io.imread(image_lst[i])
            c2_img[z_slice, :, :] = io.imread(image_lst[i + 1])
        if num_channels == 3:
            c1_img[z_slice, :, :] = io.imread(image_lst[i])
            c2_img[z_slice, :, :] = io.imread(image_lst[i + 1])
            c3_img[z_slice, :, :] = io.imread(image_lst[i + 2])
        if num_channels == 4:
            c1_img[z_slice, :, :] = io.imread(image_lst[i])
            c2_img[z_slice, :, :] = io.imread(image_lst[i + 1])
            c3_img[z_slice, :, :] = io.imread(image_lst[i + 2])
            c4_img[z_slice, :, :] = io.imread(image_lst[i + 3])
        if num_channels == 5:
            c1_img[z_slice, :, :] = io.imread(image_lst[i])
            c2_img[z_slice, :, :] = io.imread(image_lst[i + 1])
            c3_img[z_slice, :, :] = io.imread(image_lst[i + 2])
            c4_img[z_slice, :, :] = io.imread(image_lst[i + 3])
            c5_img[z_slice, :, :] = io.imread(image_lst[i + 4])
        z_slice += 1
    os.chdir(orig_dir)        
    if num_channels == 1:
        img_arr = np.copy(c2_img)
        return(img_arr)
    if num_channels == 2:
        img_arr = np.array([c1_img, c2_img], dtype = final_dtype)
        return(img_arr)
    if num_channels == 3:
        img_arr = np.array([c1_img, c2_img, c3_img], dtype = final_dtype)
        return(img_arr)
    if num_channels == 4:
        img_arr = np.array([c1_img, c2_img, c3_img, c4_img], dtype = final_dtype)
        return(img_arr)
    if num_channels  == 5:
        img_arr = np.array([c1_img, c2_img, c3_img, c4_img, c5_img], dtype = final_dtype)
        return(img_arr)

        

## move .tif files

In [66]:
''' move_tifs updated 06-20-20

This is script to organize directory full of .tif files that is created from aquiring multichannel z-stacks at multiple fields
of view in elements. This assumes that the nomenclature of the files if the left default in elemens. This is the same 
nomenclature that elements defaults to when writing images directly to .tif files. Basicall naming is is as follows:
'xy'+FOV+'z'+zslice+'c'+channel+'.tif'. This organizes all of these files so that each field of view is placed in its own sub-
directroy.

Parameters
-----------
tif_dir: string
    this should be in the form r'drive\folder\folder\folder_containing_all_tifs'. 
    
Returns
-----------
None
'''

import os
import shutil
import numpy as np


def move_tifs(tif_dir):
    
    os.chdir(tif_dir) #change to directory with .tif files

    new_dir_names = [] #temporary list to be appended with all of the first four character .tif filenames from the directory
    for filename in os.listdir(): #iterate over all files in directory
        if filename.endswith('.tif'): #check if til is .tif
            new_dir_names.append(filename[:4]) #append first four characters of .tif filename to list
    new_dir_names = list(np.unique(new_dir_names))  #make a list of only the unique entries in the list. These will be come the new subfolders for oraniziton

    for dir_name in new_dir_names: #iterate the names of new subfolders and create sub folders
        os.mkdir(dir_name)

    for i in new_dir_names: # this iterates over all of the files in directory and matches individual file with appropriate newly created sub directory
        destination_dir = tif_dir + '\\' + i #this is newly created sub-directory
        tiff_files = [] # this is list to be appended with all of the .tif files in directly. This differs from above as this contains full filenames. This is technically redundent as I could move it above and save a little bit of time but I am just feeling lazy right now
        for j in os.listdir(): # iterate over 
            if (j.endswith('.tif')): #check if tile ends
                tiff_files.append(j) #append it to the list
        for j in tiff_files: #iterated over all of the .tif files in the tiff_files list
            if (j[:4] in i): #this matches the .tif file with the appropriate sub directory
                shutil.move(tif_dir + '\\' + j, destination_dir + '\\' + j) #moves .tif file to appropriate subdirectory

## Purge directory of empty .tiff files

In [25]:
''' Sometimes in going through segmentation I come across image that is not worth saving so I just keep the segmentaiton blank. Becausse
this is part of a pipeline ---see sub_cell_segmentation.ipynb ---- it is easie to just save the blank file than stop the script and 
deal with it. For this reason I wrote quick script to iterate over segmentaiton files and delete any .tif files that are blank

Parameters
----------
segm_dir: string - directory
    This is the directory that contains the segmented images. It needs to be in the form:
        r'home\folder\folder\folder'
Returns
----------
out_lst: list
    This is a list of all of the blank .tif files. Files can be removed pretty easily with os.remove() function but I am just 
    paranoid and like to double check things before deleting them
'''

import os
from skimage import io
import numpy as np

def purge_directory(segm_dir):
    
    current_dir = os.getcwd() # record current directoyr for coming back to at end of the function
    os.chdir(cebp_segm_dir) #change to directory with .tif
    
    tiff_lst = [] #empty list to be populated with all .tif files in directory
    for i in os.listdir(): #iterate over all of the files in the directory and add the .tif files to list
        if 'tif' in i:
            tiff_lst.append(i)

        
    out_lst = [] #list to tbe populated with filenames for all blank .tif files
    for tif in tiff_lst: # iterate over all previously id'd .tif files
        img_ = io.imread(tif) #tead the .tif 
        if np.sum(img_) == 0: # sum all pixels in image and determine if blank
        #os.remove(i)
            out_lst.append(tif) #add the blank .tif files to running list 
    os.chdir(current_dir) # change working directory back to where you were at beginning of functions
    return(out_lst)

# Mics

## DCTS: shannon entropy of discreate cosine transform

In [ ]:
'''dcts updated 06-19-20

This will calculate and return the shannon entropy of the normalized discrete cosine transform. This has proven to be a good method
for finding the most in focus slice in a z-stack. This is taken taken from strategy used in doi: 10.1038/nbt.3708. 


Parameters
----------
img_ : nparray
    This is an image array on which to perform function

Returns
----------
dcts_img: float
    This is a single float value for the shannon entorpy of the discrete cosine transform of the image
'''

def dcts(img_):
    dis_cos = dct(img_) #take the discrete cosine transform of the image
    l2 = np.sqrt(np.sum(np.square(img_))) #perform l2 normalization
    inner_term = np.divide(dis_cos, l2) # normalize discrete cosine transform
    inner_term[inner_term == 0] = .0001 #it is possible that some values come outas zero. It is okay to just add a small amount to this in order to make the math work
    first_term = np.abs(inner_term) #first term in shannon entropy absolute value of the normalize DCT
    second_term = np.log2(np.abs(inner_term)) #second term of shannon entropy 
    dcts_img = np.multiply(-1, np.sum(np.multiply(first_term, second_term))) # this is final float value for entropy of DCTS
    return(dcts_img)
    #return(second_term)

## dot detector

In [ ]:

'''This is function to determine the xy coordinates for dot in image. This has proven handy for antibody staining stuff and 
determining the distance of puncta relative to CEBPb hubs.

Parameters
----------
dotty_img: nparray
    This is the original 
norm_p: float between 0 and 1
    This is the percentile by which to normalize the image. Default divides image by max intensity

The rest of the input parameters pertain to the blob_log function from skimage.feature. I have only included a subset of the 
parameters which I found are most likely to be tweeked. It can be further tuned if need be. Further documentation can be found here:
https://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.blob_log

min_sigmas: calar or sequence of scalars, optional
    the minimum standard deviation for Gaussian kernel. Keep this low to detect smaller blobs. The standard deviations of the
    Gaussian filter are given for each axis as a sequence, or as a single number, in which case it is equal for all axes.
max_sigma: scalar or sequence of scalars, optional
    The maximum standard deviation for Gaussian kernel. Keep this high to detect larger blobs. The standard deviations of the
    Gaussian filter are given for each axis as a sequence, or as a single number, in which case it is equal for all axes.
num_sigma: int, optional
    The number of intermediate values of standard deviations to consider between min_sigma and max_sigma.
threshold: float, optional.
The absolute lower bound for scale space maxima. Local maxima smaller than thresh are ignored. Reduce this to detect blobs with less intensities.

Returns:
----------
dots_y_coord: nparray
    array of y coordinates of the dots
dots_x_coord: nparray
    array of x coordinates of the dots


notes:
also of use for tuning parameters are the follow plots
plt.imshow(dotty_img_testing)
plt.scatter(dots_x_coord, dots_y_coord, s=80, facecolors='none', edgecolors='r')

'''


import numpy as np
from skimage.feature import blob_log

def ab_dot_detecter(dotty_img,  norm_p = 1, log_sigma_min = 1, log_sigma_max = 20, log_sigma_num = 5, log_thresh = .2, log_overlap = .5):
    normed_img = dotty_img /  np.quantile(dotty_img, norm_p)
    dots = blob_log(normed_img, max_sigma= log_sigma_max, min_sigma = 1 num_sigma= log_sigma_num, threshold= log_thresh, 
                   overlap = log_overlap)
    dots_y_coord = dots[:, 0]
    dots_x_coord = dots[:, 1]
    
    return(dots_y_coord, dots_x_coord)

# Segmentation


## dot_2d

In [22]:
'''dot_2d last updated :06-19-20
this is the orignal function taken from Allen institue cell segmenter:
https://github.com/AllenInstitute/aics-segmentation/blob/master/aicssegmentation/core/seg_dot.py
I have changed a couple of things but the basic function remains the same. This will take in an image in which hubs or dots need
to be segmented and return a boolean array

Paramters
----------
struct_img: nparray
    This is the original 2D image. It does not need to be normalized as above
log_sigma: float
    This is the sigma of the gaussian that is used in the LOG filter. Typical values for this range from ~1-3 but it depends on 
    the size of the objects you wish to segment
log_thresh: float
    This is the threshold value for after the LOG filter. typical values for this range from .005 - .015 but as above this depends 
    on the size and shape of the objects you wish to segment
norm_p: float between 0 and 1
    This is the percentile of the intensity value by which to normalize the input image. The default normalizes the image by the maximum
    intensity value but this may not always be the best choice. 

Returns
-----------
Returns
----------
bw: nparray, dtype == bool
    This is a segmented image of the blobs
'''


import numpy as np
from scipy import ndimage as ndi

def dot_2d(struct_img, log_sigma = 1.5, log_thresh = 0.003, norm_p = 1): 
    struct_img_normed = struct_img / np.quantile(struct_img, norm_p) #normalize the function
    bw = np.zeros(struct_img_normed.shape, dtype=bool) #set up an image array to become the mask
    responce = np.zeros_like(struct_img_normed) # set up image array to handle the output of the LOG filter
    responce = -1 * (log_sigma ** 2) * ndi.filters.gaussian_laplace(struct_img_normed, log_sigma) #This is essentially the LOG multiplied by some constants to bring into range with threhold
    bw = responce > log_thresh # Determine which values are higher than the threshold
    return(bw_

## cebp_blobs

In [19]:
'''This is script to make mask from the CEBPb blobs during adipogenesis. This requires do_2d function as a helper function. 

Parameters
----------
img: nparray
    This is the original 2d CEBPb image
gaussian_sigma: Float
    This is sigma value for gaussian kernel applied to image
log_sigma: float
    This is the sigma of the gaussian that is used in the LOG filter. Typical values for this range from ~1-3 but it depends on 
    the size of the objects you wish to segment
log_thresh: float
    This is the threshold value for after the LOG filter. typical values for this range from .005 - .015 but as above this depends 
    on the size and shape of the objects you wish to segment
norm_p: float between 0 and 1
    This is the percentile of the intensity value by which to normalize the input image. The default normalizes the image by the maximum
    intensity value but this may not always be the best choice. 

Returns
----------
bw: nparray
    This is boolean array mask of the CEBPb blobs
    
Notes
----------
-The the log_sigma and log_thresh parameters have to pretty much be tuned for each cell which is a bit of a drag.
-This also works well for DAPI heterochromatin segmentation. The log_sigma and log_thresh need to be tuned for this as well. I have found
that starting around 1.5 * the values used for the CEBPb blob maks is a good starting place. 
'''
from skimage.filters import gaussian
from skimage.morphology import remove_small_objects

def cebp_blobs( img, gaussian_sigma =1, log_sigma = 1.5, log_thresh = 0.003, norm_p = 1):
    img_blurred = gaussian(img, gaussian_sigma)
    dot_2d_out = dot_2d(img_blurred, log_sigma, log_thresh, norm_p)
    bw = remove_small_objects(dot_2d_out)
    return(bw)

## dot_2d_orig 

In [115]:
'''dot_2d_orig
this is the orignal function taken from Allen institue cell segmenter:
https://github.com/AllenInstitute/aics-segmentation/blob/master/aicssegmentation/core/seg_dot.py
It is used to detect 2d hubs within the cell. The only thing that I changed was the name of the function. The original function as
it appear from AIC is dot_2d. 

Parameters
----------
struct_img: nparray
    This is the normalized 2d input image in which to find the blobs
s2_param: list of lists of two floats
    This is a list of lists of parmeters. The first element in each list is the sigma value for the LOG. The second value for each list
    is a threshold value.

Returns
----------
bw: nparray, dtype == bool
    This is a segmented image of the blobs
'''

import numpy as np
from scipy import ndimage as ndi

def dot_2d_orig(struct_img, s2_param):
    bw = np.zeros(struct_img.shape, dtype=bool)
    for fid in range(len(s2_param)):
        log_sigma = s2_param[fid][0]
        responce = np.zeros_like(struct_img)
        responce = -1*(log_sigma**2)*ndi.filters.gaussian_laplace(struct_img, log_sigma)
        bw = np.logical_or(bw, responce>s2_param[fid][1])
    return bw

## cebp_blobs_old

In [ ]:
'''This is old version of script to identify CEBPb blobs in cells during adipogenesis. It needs dot_2d_orig as a helper function

Parameters
----------
img_: nparray
    This is the orignal image. No normalization neccesary
gaussian_sigma: float
    This is the sigma value for a gaussian filter that is used prior to finding blobs
s2_param: list of lists of two floats
    This is a list of lists of parmeters. The first element in each list is the sigma value for the LOG. The second value for each list
    is a threshold value.

Returns
----------
jnk: nparray
    This is boolean array that is segmented image. 
'''

def cebp_blobs_old(img_, gaussian_sigma = 1, s2_params = [[1.0, .03]]):
    ################################
    ## PARAMETERS for this step ##
    gaussian_smoothing_sigma = gaussian_sigma
    ################################
    # intensity normalization
    struct_img = np.divide(img_, np.max(img_))
    
    # smoothing with gaussian filter
    structure_img_smooth = gaussian(struct_img, sigma=gaussian_smoothing_sigma)
    

    
    ################################
    ## PARAMETERS for this step ##
    s2_param = s2_params
    ################################
    
    bw = dot_2d_orig(structure_img_smooth, s2_param)
        #remove small objects
    jnk = remove_small_objects(bw)
    
    return(jnk)

## Invert mask

In [ ]:
'''This is function to invert a binary mask. A lot of times it is just easier to deal with the mask images if they are in 16bit rather 
than a boolean array. For example, multiplying boolean array by 16bit fluorescent image doesnt compute. This should largely take
care of this 

Parameters
----------
orig_masked_img: nparray
    This is the original masked image. 
Returns
----------
intverted_mask
    This is the negative of the orignal mask
'''

import numpy as np 

def invert_mask(orig_masked_img):
    
    '''just to make sure that everything is in the right form I am running quick bit to guarantee it is 1s and 0s'''
    orig_masked_img = orig_masked_img.astype('uint16') #change dtype over to 16bit
    masked_img_max = np.max(orig_masked_img)  #determine the maximum pixel value
    masked_img_binary = np.divide(orig_masked_img, masked_img_max) #changes to 1s and 0s
    
    
    ones_bool = np.ones([masked_img_.shape[0], masked_img_.shape[1]]) #make a np array of ones the same shape as original mask image
    inverted_mask = np.subtract(ones_bool, masked_img_binary) #subtract orignal mask from nparray of ones
    return(inverted_mask)

## random_blob_image

In [26]:
'''updated 06-20-20
This is for creating a binary mask in which each of the segmented sub cellular objects are rotated and move around the nucleus. In 
addition to usin segmented heterochromatin regions in dapi channel, this can serve as a decent control for determing the loaction 
of puncta etc relative to hubs

Parameters
----------
nucleus_segm_img: nparray
    This is a binary mask of a single nucleus
blob_segm_img: np array
    This is a binary mask of the segmented sub cellular hubs
nucleus erosion
    This is the minimum distance from the outer edge of the nucelus that the center of each of the randomly moved blobs can be located
Returns
----------
new_blob_img: nparray
    This is a binary image in which all of the subcellular objects have been rotated and moved to random locations within the nucleus.

note
    This is note perfect. IT does not account for rotated and randomly moved subcellular objects over lapping with one anotehr. In this
    way the total area and perimeter of the randomly moved sub cellular hubs is often less than that of the original segmented image.
    There a way in which I could fix this but I don't feel like making perfect the enemy of good enough.
'''
def random_blob_image(nucleus_segm_img, blob_segm_img nucleus_erosion = 15): 
    nucleus_erode = binary_erosion(nucleus_segm_img, disk(nucleus_erosion)) #reode the segmented nucleus image so won't chooose points too close to the edge a little later on
    
    labeled_blob_segm_img = measure.label(blob_segm_img) #label the blobs within the image 
    
    new_blob_img = np.zeros([nucleus_segm_img.shape[0], nucleus_segm_img.shape[1]]).astype('uint16') #set up array of zeros with same shape as original image
    
    for labeled_blob in np.unique(labeled_blob_segm_img)[1:]: #iterate over the blobs
        indy_blob = np.array(labeled_blob_segm_img == labeled_blob).astype('uint8') #choose an individual blob to look at
        minr_b, minc_b, maxr_b, maxc_b = measure.regionprops(indy_blob)[0].bbox # find coordinates of a bounding box around the individual blob
        angle = int(np.random.randint(0, 360, 1)) #choose a random angle to ratate the blob
        indy_blob_rotate = rotate(indy_blob[minr_b:maxr_b, minc_b: maxc_b], angle) # rotate the blob
        thresh = threshold_otsu(indy_blob_rotate) #a rotation of a binary image does not return a binary image. Using otsu_threshold will turn the rotated image backingto a bianry image
        indy_blob_rotate = np.array(indy_blob_rotate > thresh).astype('uint8') # turn rotated blob back into binary image
        y_center_blob, x_center_blob = measure.regionprops(indy_blob_rotate)[0].centroid # find the center of the rotated blob
        x_center_blob = int(x_center_blob)
        y_center_blob = int(y_center_blob)
        nucleus_coordinate = np.argwhere(nucleus_erode == 1) # choose random new coordinates within the nucleus
        rand_coord_numb = int(np.random.randint(0, len(nucleus_coordinate), 1))
        new_y_center, new_x_center = nucleus_coordinate[rand_coord_numb] # record the x and y coordinates of the randomly chosen new center
        translate_y = int(new_y_center) - y_center_blob #amount to translate rotated blob
        translate_x = int(new_x_center) - x_center_blob #amount to translate rotated blob
        orig_rotated_blob_coords = np.argwhere(indy_blob_rotate == 1) # generated list of original coordinates for the rotated blob
        ycoords_rotated_orig_pos = orig_rotated_blob_coords[:, 0] #coordinates for the 
        xcoords_rotated_orig_pos = orig_rotated_blob_coords[:, 1]
        ycoords_rotated_new_pos = ycoords_rotated_orig_pos + translate_y
        xcoords_rotated_new_pos = xcoords_rotated_orig_pos + translate_x
        
        for new_y, new_x in zip(ycoords_rotated_new_pos, xcoords_rotated_new_pos):
            new_blob_img[new_y, new_x] = 1
        new_blob_img = np.multiply(nucleus_segm_img, new_blob_img)
    return(new_blob_img)

SyntaxError: invalid syntax (<ipython-input-26-ed5dbd2a0aa0>, line 20)